In [14]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [15]:
import sys
sys.path.append('./as-tp6')
from charDataset import *

ModuleNotFoundError: No module named 'charDataset'

###### Dataset Trump Tweets

On utilise l'ensemble des tweets postés par Donald Trump.

In [ ]:
data = pd.read_csv("trump.csv", dtype=str, delimiter=",", usecols=[2])
data.sample(5)

On utilise les fonctions ord et chr pour facilement convertir les caractères en valeurs numériques, et on crée les ensembles d'apprentissage et de test.

In [3]:
class Recurent(nn.Module):
    def __init__(self, tailleZ, tailleVoc, actF = nn.Sigmoid()):
        '''X et Y vecteur onehot de taille tailleVoc
        Z vecteur de stockage de taille tailleZ'''
        super(Recurent, self).__init__()
        self.XToY = nn.Linear(tailleVoc, tailleVoc)
        self.XToZ = nn.Linear(tailleVoc, tailleZ)
        self.ZToY = nn.Linear(tailleZ, tailleVoc)
        self.ZToZ = nn.Linear(tailleZ, tailleZ)
        self.actF = actF

    def forward(self, x, z):
        '''return Z, Y'''
        #print("x", x, "z", z)
        return self.actF(self.XToZ(x) + self.ZToZ(z)), self.actF(self.XToY(x) + self.ZToY(z))

In [4]:
class RecurentGated(nn.Module):
    def __init__(self, tailleZ, tailleVoc, actF = nn.Sigmoid()):
        '''X et Y vecteur onehot de taille tailleVoc
        Z vecteur de stockage de taille tailleZ'''
        super(Recurent, self).__init__()
        self.XToY = nn.Linear(tailleVoc, tailleVoc)
        self.XToZ = nn.Linear(tailleVoc, tailleZ)
        self.ZToY = nn.Linear(tailleZ, tailleVoc)
        self.ZToZ = nn.Linear(tailleZ, tailleZ)
        self.gateZ = nn.Linear(tailleVoc, tailleZ)
        self.gateY = nn.Linear(tailleVoc, tailleVoc)
        self.actF = actF

    def forward(self, x, z):
        '''return Z, Y'''
        soft = torch.nn.Softmax(-1)
        return soft(self.XtoZ(x)) * self.actF(self.gateZ(x)) + self.actF(self.ZtoZ(z)) * (1-self.actF(self.gateZ(x))),\
    self.actF(self.XtoY(x)) * self.actF(self.gateY(x)) + self.actF(self.ZtoY(z)) + (1 - self.actF(self.gateY(x)))

In [5]:
fData = "./as-tp6/train_data.tx"
fVoc = "./as-tp6/vocab.tx"
ten = torch.load(fData)
rawVoc = torch.load(fVoc)

In [6]:
class bidict(dict):
    def __init__(self, *args, **kwargs):
        super(bidict, self).__init__(*args, **kwargs)
        self.inverse = {}
        for key, value in self.items():
            self.inverse.setdefault(value,[]).append(key) 

    def __setitem__(self, key, value):
        if key in self:
            self.inverse[self[key]].remove(key) 
        super(bidict, self).__setitem__(key, value)
        self.inverse.setdefault(value,[]).append(key)        

    def __delitem__(self, key):
        self.inverse.setdefault(self[key],[]).remove(key)
        if self[key] in self.inverse and not self.inverse[self[key]]: 
            del self.inverse[self[key]]
        super(bidict, self).__delitem__(key)

In [7]:
voc = bidict(rawVoc)
voc[' '] = 1
voc['0'] = 0
voc['.'] = 35
voc['z'] = 33
voc['y'] = 34
voc['y'] = 34
voc['w'] = 32

In [8]:
voc

{' ': 1,
 '.': 35,
 '0': 0,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '7': 7,
 '8': 8,
 '_': 9,
 'a': 10,
 'b': 11,
 'c': 12,
 'd': 13,
 'e': 14,
 'f': 15,
 'g': 16,
 'h': 17,
 'i': 18,
 'j': 19,
 'k': 20,
 'l': 21,
 'm': 22,
 'n': 23,
 'o': 24,
 'p': 25,
 'q': 26,
 'r': 27,
 's': 28,
 't': 29,
 'u': 30,
 'v': 31,
 'w': 32,
 'x': 32,
 'y': 34,
 'z': 33}

In [50]:
class RNNMono(nn.Module):
    def __init__(self, recur, stop):
        super(RNNMono, self).__init__()
        self.recur = recur
        self.stop = stop
    
    def forward(self, boot, z):
        for i in boot:
            z, x = self.recur(i,z)
        r = []
        r.append(x)
        while (x.max(-1)[1].data != self.stop.max(-1)[1]).all() :
            z, x = self.recur(x,z)
            #print(voc.inverse[x.max(-1)[1].data[-1]])
            r.append(x)
        return r, z

In [10]:
def sequencer(x, end):
    r = []
    for i in x :
        r.append(i)
        if i == end :
            yield r
            r = []

In [11]:
for i in sequencer(ten[:1000], 35):
    print(code2char(i, voc))

dave aneckstein4 simmons research4 an ezperian company they have not been charged or formally arrested5 iran isnt making an atomic bomb4 not at all4 chave.
 said monday5 the japanesemade tin robots have blocky heads and moveable arms and legs5 if they could no longer be the nominees4 then they would be pundits of the first order men with credibility on oval office matters by dint of once sitting in the chair themselves5 free challenge kits have a cd and brochure from dr5 ian4 menu and fitness advice and a pedometer to count steps5 the world motor sport council received statements from fernando alonso4 lewis hamilton and pedro de la rosa stating categorically no ferrari information had been used by mclaren4 and that no confidential data had been passed to the team5 the prime minister said the first citi.


In [ ]:
torch.manual_seed(1)
zLen = 1000
vocLen = 36
recur = Recurent(zLen, vocLen)

endN = torch.LongTensor([vocLen-1])
end_onehot = torch.FloatTensor(*endN.size(), vocLen).zero_()
end_onehot.scatter_(1, torch.unsqueeze(endN, 1), 1.)

m = RNNMono(recur, end_onehot)
ml = nn.MSELoss()
ite = 500
opt = optim.SGD(m.parameters(), lr=1e-3)

lossHisto = []
scoreHisto = []

for i,seq in enumerate(sequencer(ten, 35)):
    if i > ite :
        break
    if i%(ite/10) == 0:
        print("Iteration", i)
    
    xN = torch.LongTensor(seq[:30])
    x_onehot = torch.FloatTensor(*xN.size(), vocLen).zero_()
    x_onehot.scatter_(1, torch.unsqueeze(xN, 1), 1.)
    x = autograd.Variable(x_onehot.type(torch.FloatTensor))
    
    yN = torch.LongTensor(seq[30:])
    y_onehot = torch.FloatTensor(*yN.size(), vocLen).zero_()
    y_onehot.scatter_(1, torch.unsqueeze(yN, 1), 1.)
    y = autograd.Variable(y_onehot.type(torch.FloatTensor))
    
    z = autograd.Variable(torch.zeros(zLen).type(torch.FloatTensor))
    
    f,_ = m.forward(x, z)
    print(f, y)
    loss = ml.forward(f, y)
    loss.backward()
    opt.step()
    
    lossHisto.append(loss.data.mean())
    ypred = torch.max(f, 1)[1]
    scoreHisto.append(torch.eq(ypred.data, yN).float().mean())
    
plt.plot(lossHisto)
plt.ylabel('loss')
plt.xlabel('nb batch traited')
plt.show()
plt.plot(scoreHisto)
plt.ylabel('score')
plt.xlabel('nb batch traited')
plt.show()